In [14]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager

def download_car(dest_path):
    with open(os.path.join(dest_path, 'links.txt'), 'r') as file:
        arr_url = file.read().splitlines()

    options = Options()
    options.headless = True

    # Thiết lập đường dẫn mặc định cho việc tải xuống
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", dest_path)
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")

    driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)
    driver.set_page_load_timeout(30)  # Set page load timeout to 30 seconds
    driver.implicitly_wait(10)        # Set implicit wait time to 10 seconds

    df_links = pd.DataFrame(columns=['Original Link', 'Found Link', 'Link Text', 'View Listing'])
    found_links_file = os.path.join(dest_path, 'foundlink.txt')

    with open(found_links_file, 'w') as file:
        file.write('')

    for original_url in arr_url:
        driver.get(original_url)
        WebDriverWait(driver, 10)
        elements = driver.find_elements(By.XPATH, "//a[@href]")
        for element in elements:
            href = element.get_attribute("href")
            text = element.text
            df_links = df_links.append({
                'Original Link': original_url,
                'Found Link': href,
                'Link Text': text,
                'View Listing': ''
            }, ignore_index=True)

            with open(found_links_file, 'a') as file:
                file.write(href + '\n')

    with open(found_links_file, 'r') as file:
        found_urls = file.read().splitlines()

    for found_url in found_urls:
        driver.get(found_url)
        WebDriverWait(driver, 10)
        try:
            view_listing_element = driver.find_element(By.CSS_SELECTOR, '#status-banner-analytics > a')
            view_listing_href = view_listing_element.get_attribute('href')
            df_links.loc[df_links['Found Link'] == found_url, 'View Listing'] = view_listing_href
        except Exception as e:
            print(f'No view listing found for {found_url}: {e}')

    driver.quit()
    df_links.to_csv(os.path.join(dest_path, 'collected_links.csv'), index=False)

if __name__ == '__main__':
    cwd = os.getcwd()
    dest_path = os.path.join(cwd, 'data_source')
    os.makedirs(dest_path, exist_ok=True)
    download_car(dest_path)


ValueError: Timeout value connect was <object object at 0x0000029FC4FB8960>, but it must be an int, float or None.

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

file_path = r"D:\GitHub\Work-Experiences\Tuan Loc Commodities\2. Scrapping\Links.txt"

with open(file_path, 'r') as file:
    arr_urls = file.read().splitlines()

# Khởi tạo trình duyệt
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

# Tạo DataFrame để lưu trữ dữ liệu
df_links = pd.DataFrame(columns=['Original Link', 'Found Link', 'Link Text', 'View Listing'])

# Duyệt qua từng liên kết và thu thập dữ liệu
found_links_file = r"D:\GitHub\Work-Experiences\Tuan Loc Commodities\2. Scrapping\foundlink.txt"

with open(found_links_file, 'w') as file:
    file.write('')

for original_url in arr_urls:
    driver.get(original_url)
    elements = driver.find_elements(By.XPATH, "//a[@href]")
    for element in elements:
        href = element.get_attribute("href")
        text = element.text
        df_links = pd.concat([df_links, pd.DataFrame(columns=['Original Link', 'Found Link', 'Link Text', 'View Listing'])])

        with open(found_links_file, 'a') as file:
            file.write(href + '\n')

with open(found_links_file, 'r') as file:
    found_urls = file.read().splitlines()

for found_url in found_urls:
    driver.get(found_url)
    try:
        view_listing_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#status-banner-analytics > a'))
        )
        view_listing_href = view_listing_element.get_attribute('href')
        df_links.loc[df_links['Found Link'] == found_url, 'View Listing'] = view_listing_href
    except Exception as e:
        print(f'Không tìm thấy liên kết xem danh sách cho {found_url}: {e}')

# Lưu DataFrame vào tệp CSV
df_links.to_csv(df_links.csv, index=False)

# Đóng trình duyệt khi hoàn tất
driver.quit()

# In DataFrame ra console để kiểm tra
print(df_links)


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Khởi tạo trình duyệt
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

file_path = r"D:\GitHub\Work-Experiences\Tuan Loc Commodities\2. Scrapping\Links.txt"

with open(file_path, 'r') as file:
    arr_urls = file.read().splitlines()

# Tạo DataFrame để lưu trữ dữ liệu
df_links = pd.DataFrame(columns=['Original Link', 'Found Link', 'Link Text', 'View Listing'])

found_links_file = r"D:\GitHub\Work-Experiences\Tuan Loc Commodities\2. Scrapping\foundlink.txt"

with open(found_links_file, 'w') as file:
    file.write('')

for original_url in arr_urls:
    driver.get(original_url)
    elements = driver.find_elements(By.CSS_SELECTOR, "#search-form > div > div:nth-child(3) > div > div.listings-table div a[href]")
    for element in elements:
        try:
            href = element.get_attribute("href")
            text = element.text
            df_links = pd.concat([df_links, pd.DataFrame(columns=['Original Link', 'Found Link', 'Link Text', 'View Listing'])])

            if href is not None:  # Add this check
                with open(found_links_file, 'a') as file:
                    file.write(href + '\n')
        except Exception as e:
            print(f"Error occurred while processing element: {e}")

with open(found_links_file, 'r') as file:
    found_urls = file.read().splitlines()

for found_url in found_urls:
    driver.get(found_url)
    time.sleep(1)  # Adding a small delay for stability, you might adjust it
    try:
        view_listing_element = driver.find_element(By.CSS_SELECTOR, '#status-banner-analytics > a')
        view_listing_href = view_listing_element.get_attribute('href')
        df_links.loc[df_links['Found Link'] == found_url, 'View Listing'] = view_listing_href
    except Exception as e:
        print(f'Không tìm thấy liên kết xem danh sách cho {found_url}: {e}')

# Lưu DataFrame vào tệp CSV
df_links.to_csv('df_links.csv', index=False)

# Đóng trình duyệt khi hoàn tất
driver.quit()

# In DataFrame ra console để kiểm tra
print(df_links)
